# Dsparse

DSparseTensor: Distributed Sparse Tensor Example

Demonstrates:
- Creating DSparseTensor with automatic partitioning
- SparseTensor.partition() and DSparseTensor.gather()
- Distributed matrix-vector product
- Distributed solve

In [ ]:
sys.path.insert(0, '..')

import torch
import os


from torch_sla import DSparseTensor, SparseTensor

## Helper Functions and Classes

In [ ]:
def create_2d_poisson(nx: int, ny: int, dtype=torch.float64):
    """Create 2D Poisson matrix with 5-point stencil."""
    N = nx * ny
    idx = torch.arange(N)
    i, j = idx // nx, idx % nx
    
    entries = [
        (idx, idx, torch.full((N,), 4.0, dtype=dtype)),
        (idx[i > 0], idx[i > 0] - nx, torch.full(((i > 0).sum(),), -1.0, dtype=dtype)),
        (idx[i < ny-1], idx[i < ny-1] + nx, torch.full(((i < ny-1).sum(),), -1.0, dtype=dtype)),
        (idx[j > 0], idx[j > 0] - 1, torch.full(((j > 0).sum(),), -1.0, dtype=dtype)),
        (idx[j < nx-1], idx[j < nx-1] + 1, torch.full(((j < nx-1).sum(),), -1.0, dtype=dtype)),
    ]
    
    row = torch.cat([e[0] for e in entries])
    col = torch.cat([e[1] for e in entries])
    val = torch.cat([e[2] for e in entries])
    
    return val, row, col, (N, N)